In [180]:
from collections import defaultdict
from queue import Queue, PriorityQueue
import numpy as np
import pandas as pd

class MemoryManager:
    def __init__(self, mem_size, policy, page_size=0):
        self.mem_size = mem_size
        self.policy = policy

        if self.policy == 'partitioned':
            self.memory = [('SO', 0, 0), ('END', self.mem_size, self.mem_size)]

        elif self.policy == 'paged':
            assert page_size != 0 and mem_size % page_size == 0 
            self.n_pages = int(mem_size/page_size)
            self.memory = [{'job':None, 'used': 0} for _ in range(self.n_pages)]
            self.page_size = page_size
            self.remaining_pages = self.n_pages

    def allocate(self, job, size):
        if self.policy == 'partitioned':
            for i in range(len(self.memory) - 1):
                end_last_part = self.memory[i][2]
                start_next_part = self.memory[i+1][1]
                part_size = start_next_part - end_last_part 

                if size <= part_size:
                    self.memory.insert(i+1, (job, end_last_part, end_last_part + size))
                    return True
            return False

        elif self.policy == 'paged':
            remaining_size = size
            if self.remaining_pages * self.page_size >= size:
                for i in range(len(self.memory)):
                    if self.memory[i]['job'] == None:
                        self.memory[i]['job'] = job
                        self.memory[i]['used'] = min(self.page_size, remaining_size) 
                        remaining_size -= self.page_size
                        self.remaining_pages -= 1
                        if remaining_size <= 0: break
                return True
            else:
                return False

                        

    def deallocate(self, job):
        if self.policy == 'partitioned':
            for i in range(len(self.memory)):
                if job == self.memory[i][0]:
                    self.memory.pop(i)
                    return

        elif self.policy == 'paged':
            for i in range(len(self.memory)):
                if self.memory[i]['job'] == job:
                    self.memory[i]['job'] = None 
                    self.memory[i]['used'] = 0
                    self.remaining_pages += 1

    def print_memory(self):
        if self.policy == 'partitioned':
            print(self.memory)

        elif self.policy == 'paged':
            df = pd.DataFrame.from_dict(self.memory)
            df.index.rename('page', inplace=True)            
            display(df)
            print('Free pages: ',self.remaining_pages)



class Simulation:
    def __init__(self, job_mix, time_unit, it_max, loader_policy, max_multiprog, mem_size, mem_policy, time_offset = 0,page_size=0):
        self.job_mix = job_mix
        self.time_unit = time_unit
        self.it_max = it_max
        self.max_multiprog = max_multiprog
        self.time_offset = time_offset

        if(loader_policy == 'FIFO'):  
            self.job_queue = Queue()
        elif(loader_policy == 'Priority'):
            self.job_queue = PriorityQueue()
        
        self.memoryManager = MemoryManager(mem_size, mem_policy, page_size)

        self.buffer = []
        self.executing = []
        self.output = defaultdict(dict)
        self.t = 0
        self.it = 0
        self.eps = 0.000001



    def queue_job(self, job, load):
        self.job_queue.put((load, job))
        self.output[job]['t_allocated'] = self.t

        #self.output[job]['arrival'] = self.t
        #self.output[job]['R'] = load

    def buffer_job(self, job, load, size):
        self.buffer.append((job, load, size))
        self.output[job]['arrival'] = self.t
        self.output[job]['R'] = load
        

    def loader(self):
        #print('time t=', self.t)

        #execute
        for i in range(len(self.executing)):
            self.executing[i]['load'] -= self.time_unit/len(self.executing)

        #finish
        aux_list = []
        for i in range(len(self.executing)):
            if(self.executing[i]['load'] <= self.eps):
                # account for time unit being larger than load execution when multiprogrammed
                #print(' finish load =',  self.executing[i]['load'])

                self.memoryManager.deallocate(self.executing[i]['job'])
                self.memoryManager.print_memory()
                self.output[self.executing[i]['job']]['finish'] = self.t + self.executing[i]['load']
            else:
                aux_list.append(self.executing[i])
        self.executing = aux_list

        #allocate
        for job, load, size in self.buffer:
            alloc_result = self.memoryManager.allocate(job, size)
            if alloc_result == True:
                self.memoryManager.print_memory()
                self.queue_job(job, load)
                self.buffer.remove((job,load,size))
        
        #load 
        while(  (len(self.executing) < self.max_multiprog or self.max_multiprog == -1) and not self.job_queue.empty()):

            load, job = self.job_queue.get()
            #print(' load', job)
            self.executing.append({'job':job, 'load':load})

            self.output[job]['start'] = self.t

        #print(' ', self.executing)


    def run(self):
        self.t = 0 + self.time_offset
        self.it = 0

        for self.it in range(self.it_max):
            self.t = self.it*self.time_unit + self.time_offset
            
            for job, arrival, load, size in self.job_mix:
                if self.t == arrival :
                    self.buffer_job(job, load, size)

            self.loader()

        self.df_output = pd.DataFrame.from_dict(self.output, orient='index')
        self.df_output['T'] = self.df_output['finish'] - self.df_output['arrival']
        self.df_output['W'] = self.df_output['T']/self.df_output['R']
        self.df_output['']
        self.df_output.index.rename('Job', inplace=True)

        return self.df_output



            

In [181]:

job_mix = [
    ('1', 0, 0.3, 100),
    ('2', 0.2, 0.5, 100),
    ('3', 0.4, 0.1, 100),
    ('4', 0.5, 0.4, 100),
    ('5', 0.8, 0.1, 100)
]

# job_mix = [
#     (1, 0, 120),
#     (2, 10, 60),
#     (3, 25, 15),
# ]

#display(pd.DataFrame(job_mix, columns=['Job', 'Chegada', 'Tempo de Execução']))


In [182]:
fifo_sim = Simulation(job_mix, time_unit=0.01, it_max=20000, loader_policy='FIFO', max_multiprog=-1, mem_policy='paged', mem_size=225, page_size=75)
display(fifo_sim.run()[['arrival', 'start', 'finish', 'T', 'R', 'W', 't_allocated']])

,job,used
page,,
0,1,75
1,1,25
2,None,0


Free pages:  1


,job,used
page,,
0,None,0
1,None,0
2,None,0


Free pages:  3


,job,used
page,,
0,2,75
1,2,25
2,None,0


...

,job,used
page,,
0,5,75
1,5,25
2,None,0


Free pages:  1


,job,used
page,,
0,None,0
1,None,0
2,None,0


Free pages:  3


,arrival,start,finish,T,R,W,t_allocated
Job,,,,,,,
1,0.0,0.0,0.3,0.3,0.3,1.0,0.0
2,0.2,0.3,0.8,0.6,0.5,1.2,0.3
3,0.4,0.8,0.9,0.5,0.1,5.0,0.8
4,0.5,0.9,1.3,0.8,0.4,2.0,0.9
5,0.8,1.3,1.4,0.6,0.1,6.0,1.3


In [112]:
fifo_sim = Simulation(job_mix, time_unit=0.01, it_max=20000, loader_policy='FIFO', max_multiprog=1, mem_size=300, mem_policy='partitioned')
display(fifo_sim.run()[['arrival', 'start', 'finish', 'T', 'R', 'W', 'allocated']])

[('SO', 0, 0), (1, 0, 100), ('END', 300, 300)]
[('SO', 0, 0), (1, 0, 100), (2, 100, 200), ('END', 300, 300)]
[('SO', 0, 0), (3, 0, 100), (2, 100, 200), ('END', 300, 300)]
[('SO', 0, 0), (3, 0, 100), (2, 100, 200), (4, 200, 300), ('END', 300, 300)]
[('SO', 0, 0), (3, 0, 100), (5, 100, 200), (4, 200, 300), ('END', 300, 300)]


,arrival,start,finish,T,R,W,allocated
1,0.0,0.0,0.3,0.3,0.3,1.0,0.0
2,0.2,0.3,0.8,0.6,0.5,1.2,0.2
3,0.4,0.8,0.9,0.5,0.1,5.0,0.4
4,0.5,0.9,1.3,0.8,0.4,2.0,0.5
5,0.8,1.3,1.4,0.6,0.1,6.0,0.8


In [175]:
# cdefgh
# 336528

[a, b, c, d, e, f, g, h] = '10336528'


job_mix_prova = [
    ('1', int(f'{c}{d}'), int(f'39{d}'), 100),
    ('2', int(f'3{d}{e}'), int(f'{c}3'), 150),
    ('3', int(f'{c}{d}'), int(f'{h}8'), 30),
    ('4', int(f'1{f}{g}'), int(f'4{f}'), 70),
    ('5', int(f'{g}{h}'), int(f'3{g}{g}'), 80),
    ('6', int(f'2{h}{a}'), int(f'{e}01'), 90)
]

for x in job_mix_prova:
    print(x)

('1', 33, 393, 100)
('2', 336, 33, 150)
('3', 33, 88, 30)
('4', 152, 45, 70)
('5', 28, 322, 80)
('6', 281, 601, 90)


In [185]:
sim = Simulation(job_mix_prova, time_offset=28, time_unit=0.1, it_max=200000, loader_policy='FIFO', max_multiprog=-1, mem_policy='paged', mem_size=225, page_size=75)
display(sim.run()[['arrival', 'start', 'finish', 'T', 'R', 'W', 't_allocated']])

,job,used
page,,
0,5,75
1,5,5
2,None,0


Free pages:  1


,job,used
page,,
0,5,75
1,5,5
2,3,30


Free pages:  0


,job,used
page,,
0,5,75
1,5,5
2,None,0


...

,job,used
page,,
0,2,75
1,2,75
2,None,0


Free pages:  1


,job,used
page,,
0,None,0
1,None,0
2,None,0


Free pages:  3


,arrival,start,finish,T,R,W,t_allocated
Job,,,,,,,
5,28.0,28.0,483.0,455.0,322,1.413043,28.0
1,33.0,483.0,876.0,843.0,393,2.145038,483.0
3,33.0,33.0,209.0,176.0,88,2.000000,33.0
4,152.0,209.0,299.0,147.0,45,3.266667,209.0
6,281.0,876.0,1477.0,1196.0,601,1.990017,876.0
2,336.0,1477.0,1510.0,1174.0,33,35.575758,1477.0


In [34]:
def run_ensaios(job_mix):
    print('Política: FIFO, Máximo grau de multiprog.: 1')
    fifo = Simulation(job_mix, time_unit=0.1, it_max=1000000, loader_policy='FIFO', max_multiprog=1)
    display(fifo.run()[['arrival', 'start', 'finish', 'T', 'R', 'W']])

    print('Política: Job mais curto primeiro, Máximo grau de multiprog.: 1')
    priority = Simulation(job_mix, time_unit=0.1, it_max=1000000, loader_policy='Priority', max_multiprog=1)
    display(priority.run()[['arrival', 'start', 'finish', 'T', 'R', 'W']])

    print('Política: FIFO, Máximo grau de multiprog.: 6')
    multiprog = Simulation(job_mix, time_unit=0.1, it_max=1000000, loader_policy='FIFO', max_multiprog=6)
    display(multiprog.run()[['arrival', 'start', 'finish', 'T', 'R', 'W']])

In [24]:
run_ensaios(job_mix_prova)

Política: FIFO, Máximo grau de multiprog.: 1


,arrival,start,finish,T,R,W
1,33.0,350.0,743.0,710.0,393,1.806616
2,336.0,1477.0,1510.0,1174.0,33,35.575758
3,33.0,743.0,831.0,798.0,88,9.068182
5,28.0,28.0,350.0,322.0,322,1.000000
4,152.0,831.0,876.0,724.0,45,16.088889
6,281.0,876.0,1477.0,1196.0,601,1.990017


Política: Job mais curto primeiro, Máximo grau de multiprog.: 1


,arrival,start,finish,T,R,W
1,33.0,516.0,909.0,876.0,393,2.229008
2,336.0,350.0,383.0,47.0,33,1.424242
3,33.0,428.0,516.0,483.0,88,5.488636
5,28.0,28.0,350.0,322.0,322,1.000000
4,152.0,383.0,428.0,276.0,45,6.133333
6,281.0,909.0,1510.0,1229.0,601,2.044925


Política: FIFO, Máximo grau de multiprog.: 6


,arrival,start,finish,T,R,W
1,33.0,33.0,1230.168333,1197.168333,393,3.046230
2,336.0,336.0,474.885000,138.885000,33,4.208636
3,33.0,33.0,363.193333,330.193333,88,3.752197
5,28.0,28.0,1078.168333,1050.168333,322,3.261392
4,152.0,152.0,346.500000,194.500000,45,4.322222
6,281.0,281.0,1510.085000,1229.085000,601,2.045067
